In [6]:
import os
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import os, time
from tqdm import tqdm
import numpy as np
import pandas as pd
import cv2
from tqdm import tqdm_notebook, tqdm # Iteration visualization
from PIL import Image
import matplotlib.pyplot as plt
%matplotlib inline
import random


In [7]:
def from_yolo_to_cor(box, img_h, img_w): 
    x1, y1 = int((box[0] - box[2]/2)*img_w), int((box[1] - box[3]/2)*img_h)
    x2, y2 = int((box[0] + box[2]/2)*img_w), int((box[1] + box[3]/2)*img_h)
    return x1, y1, x2, y2
    
def cropping_images(path, new_path,bbox):
    # import image
    f_image = Image.open(path)
    
    #Bounding box cordinates
    left, top, right, bottom = from_yolo_to_cor(bbox, f_image.size[1], f_image.size[0])
#     plt.figure(figsize = (12, 8))    
#     img = cv2.imread(path)
#     plt.subplot(1, 2,1); plt.imshow(img);
     
    # Crop the image
    f_image = f_image.crop((left, top, right, bottom))
    try:
        f_image.save(new_path, 'jpeg')
    except:
        #print(f'Used convertor for {new_path}')
        f_image = f_image.convert('RGB')
        f_image.save(new_path, 'jpeg')

In [ ]:
# directory = 'data/venus/women/tops/sleeveless/final_images'
# os.mkdir('data/venus/women/tops/sleeveless/final_images')

In [11]:
directory = 'data/venus/women/tops/cold_shoulder/final_images'
all_files = os.listdir(directory + '/bbox_txt')
all_files.remove('.DS_Store')

In [12]:
for idx, f in enumerate(all_files):
    bbox_path = directory + '/bbox_txt/' + f
    img_path = directory + '/images/' + f[:-3] + 'jpg'
    bbox_file = open(bbox_path, 'r')
    bbox_data = bbox_file.readlines()
    if len(bbox_data) != 3:
        print(idx, f)
        break
    bbox_data = np.array([[float(box) for box in bbox.split()] for bbox in bbox_data])
    assert len(bbox_data) == 3, print(f'The length for {f} is not correct')
    assert bbox_data[0][0] != bbox_data[1][0] != bbox_data[2][0], print(f'There are not three classes in {f}')
    for box in bbox_data:
        new_path = directory + '/Cropped_images/Face/' + f[:-9] + '.jpg'
        if box[0] == 1:
            new_path = new_path.replace('Cropped_images/Face/','Cropped_images/Upper_cloth/')
        elif box[0] == 2:
            new_path = new_path.replace('Cropped_images/Face/','Cropped_images/Lower_cloth/')
        if int(box[0]) not in [0, 1, 2]: 
            raise ValueError(f'The class of {box[0]} is not within the pre-defined classes for this image {f}')
        cropping_images(img_path, new_path,box[1:])